To do:

- https://hal.science/hal-01586547/document
- optional Web UI

In [8]:
import CoolProp.CoolProp as cp
from math import pi, log10

In [9]:
# PropsSI uses SI Units https://en.wikipedia.org/wiki/International_System_of_Units 
# PropsSI available properties http://www.coolprop.org/coolprop/HighLevelAPI.html#table-of-string-inputs-to-propssi-function 

class fluid_properties:
    def __init__(self, fluid, temperature, pressure, mass_flowrate):
        temperature_si = temperature + 273.15
        pressure_si = pressure * 100000
        
        self.density = cp.PropsSI('D','T',temperature_si,'P',pressure_si,fluid) # kg/m3
        self.dynamic_viscosity = cp.PropsSI('V','T',temperature_si,'P',pressure_si,fluid) # Pa.s
        self.specific_heat = cp.PropsSI('C','T',temperature_si,'P',pressure_si,fluid) # J/kg.K
        self.thermal_conductivity = cp.PropsSI('CONDUCTIVITY','T',temperature_si,'P',pressure_si,fluid) # W/m.K
        self.pr_number = cp.PropsSI('PRANDTL','T',temperature_si,'P',pressure_si,fluid)
        self.phase = cp.PhaseSI('T',temperature_si,'P',pressure_si,fluid)

        self.mass_flowrate = mass_flowrate
        self.volumetric_flowrate = self.mass_flowrate / self.density

In [10]:
# Pipe and fluid parameters calculation

class pipe_properties:
    def __init__(self, od, thickness, length, roughness):
        self.id = od - (2 * thickness) #m
        self.length = length #m
        self.flow_area = 0.25 * pi * (self.id ** 2) # m2
        self.relative_roughness = roughness / self.id

In [11]:
# Gnielinkski correlation https://en.wikipedia.org/wiki/Nusselt_number#Gnielinski_correlation
# Friction factor https://en.wikipedia.org/wiki/Darcy_friction_factor_formulae#Haaland_equation 

def gnielinski(pipe, fluid):
    fluid_velocity = fluid.volumetric_flowrate / pipe.flow_area # m/s
    re_number = fluid.density * fluid_velocity * pipe.id / fluid.dynamic_viscosity
    f = (1 / (-1.8 * log10(((pipe.relative_roughness / 3.7) ** 1.11) + (6.9 / re_number)))) ** 2
    nu_number = ((f / 8) * (re_number - 1000) * fluid.pr_number) / (1 + 12.7 * ((f / 8) ** 0.5) * ((fluid.pr_number ** (2/3)) - 1))
    htc = nu_number * fluid.thermal_conductivity / pipe.id
    return htc

In [12]:
# Pipe inputs, dimensions in m
# Nominal pipe sizes https://en.wikipedia.org/wiki/Nominal_Pipe_Size 

pipe_od = 0.01029
pipe_thickness = 0.000889
pipe_length = 4
pipe_roughness = 45e-6

pipe1 = pipe_properties(pipe_od, pipe_thickness, pipe_length, pipe_roughness)

In [13]:
# Fluid inputs
# Available fluids http://www.coolprop.org/fluid_properties/PurePseudoPure.html#list-of-fluids

fluid = 'HeavyWater'
fluid_temperature = 270 # C
fluid_pressure = 150 # bar
fluid_mass_flowrate = 0.68 # kg/s

fluid1 = fluid_properties(fluid, fluid_temperature, fluid_pressure, fluid_mass_flowrate)

In [14]:
test = gnielinski(pipe1, fluid1)
print(test)

219204.24624934728
